# DSC 80: Project 02

### Checkpoint Due Date: Thursday, April 25, 11:59:59 PM

### Final Due Date: Thursday, May 02, 11:59:59 PM

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `projectXX.py` file, that will be imported into the current notebook. This code will be autograded.
* The project also has free response questions. To answer the free response questions, edit the markdown cell where specified (as in DSC 10). Submission of the project include uploading a pdf of this notebook to gradescope for manual grading.

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Do not change the free response cells outside the horizontal lines**
- The format of the cells will be used in grading the free response questions.


**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the HW! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `projectXX.py` (much like we do in the notebook).
- Always document your code!

**Tips for writing the free response questions**:
- You should treat the notebook as a final report for the assignment, containing conclusions and answers to open ended questions that are graded.
- Upon submitting the notebook, there should not be extraneous code in the notebook (e.g. any debugging code). You should only have your answers the the questions, and the necessary code and corresponding output data that serves as evidence for your responses.
- Generally, the free response questions will involve you *using* the functions defined in your `.py` file to justify portions of your argument.
- They should not be long, verbose answers! Typically a short paragraph will do.

## Checkpoint Instructions

* The checkpoint requires you to turn in **questions 1-5**; 
* The checkpoint will be graded for *approximate* correctness: easier than the final tests; harder than the doctests.
* The checkpoint will **count toward extra-credit grade** at the end of the course.

## Style Component (required)

* The project will also be graded for coding style as well.
* **Comments**: you should comment your code; the comments should explain what the code is doing (or plans to do). The comments should explain the main steps of your algorithm in plain English.
* **Helper Functions**: You are encouraged to create other functions in the `py` file that your solutions call. You should use helper functions when:
    - it makes your code more readable,
    - your code has more than 2-3 "logical steps" (use a helper function per step).
    - you see an opportunity to reuse code between a number of problems (or on future assignments)

In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import project02 as proj

In [86]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

# An Investigation into Flight Delays

### The flights dataset

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). There are data for the years 1987 - 2018. See the description of columns in `data/columns.txt`.

This project will look at a single year (2015) to keep the analysis "simple", which is available at the URL below (*NOT* on the data.gov site).


* To download the flights dataset to your computer, use [this link](https://s3-us-west-1.amazonaws.com/dsc80-sp19-data/project02/flight-delays.zip), unzip the file, and place `flights.csv` in your project directory.

* To download the dataset on `datahub.ucsd.edu` (this works on your computer as well!):
    - Open the terminal in datahub ("new > Terminal")
    - Change the directory to where you want your data (e.g. `cd [ASSIGNMENT_PATH]/data`)
    - Download the unzipped dataset using these commands:
        1. `wget https://s3-us-west-1.amazonaws.com/dsc80-sp19-data/project02/flight-delays.zip`
        2. `unzip flight-delays.zip`
    - `flights.csv` should be in the directory.
    
**NOTE: The unzipped files must be in the `project02/data` directory in order for the doctests to work!**

### Creating your datasets

**Question 1**

The flights dataset for 2015 is not small (~600MB). While you could likely load the entire dataset into Pandas on your laptop, if you wanted to work with more than one year, this would quickly become difficult (the data is available for 1987-2018). Therefore, we will filter down the dataset into two smaller files without ever reading the larger dataset fully into memory. We are going to create two smaller datasets:

1. All flights arriving or departing from San Diego Airport in 2015.
2. All flights flown by either JetBlue or Southwest Airline in 2015.

---

To do this, you are going to use the `chunksize=N` keyword in Pandas `read_csv` to read the flights dataset in blocks of `N` lines. When you use this keyword argument, `pd.read_csv(fp, chunksize=N)` becomes a *iterator* that iterates through dataframes of length N until you have reached the end of the dataset. A typical pattern looks like:
```
L = pd.read_csv(filepath, chunksize=1000)
for df in L:
    process(df)
```
Where each `df` is a dataframe of length 1000. 

The processing you are going to do is:
1. Iterate through the dataset, chunk-by-chunk,
2. Filtering out rows of each chunk
3. Incrementally add to a filtered csv file (since the data is perhaps too big to keep in memory). Keep in mind, if you want to keep writing to the same file, the mode='a' keyword in the `.to_csv` method can be helpful when calling it in the loop (a stands for 'append')

---

Write two functions that create the datasets below, using the 'chunking' pattern described above. Your functions should use `chunksize` of 10000.
1. `get_san` which takes in a filepath containing all flights and an filepath where filtered dataset #1 is written. The function should return `None`.
1. `get_jb_sw` which takes in a filepath containing all flights and an filepath where filtered dataset #2 is written. The function should return `None`.

*Remark 1*: Gradescope autograding servers are quite small and can't load this dataset into memory -- so your code that reads in the large `flights.csv` dataset *must* work with chunks of the dataset one at a time to pass!

*Remark 2*: You can check your work using the datasets included in the zip file!

In [87]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'santest.tmp')

L = pd.read_csv(infp, chunksize = 10)

for df in L:
    from_san = df['ORIGIN_AIRPORT'] == 'SAN'
    to_san = df['DESTINATION_AIRPORT'] == 'SAN'
    san = np.logical_or(from_san, to_san)
    
    san_df = df.loc[san]
    with open(outfp, 'a') as f:
        san_df.to_csv(f, index=False, header = f.tell() == 0)

os.remove(outfp)

In [88]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'santest.tmp')
proj.get_san(infp, outfp)
df = pd.read_csv(outfp)
os.remove(outfp)
df.shape

(53, 31)

In [89]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'jbswtest.tmp')

L = pd.read_csv(infp, chunksize = 10000)

for df in L:
    jb = df['AIRLINE'] == 'B6'
    sw = df['AIRLINE'] == 'WN'
    jb_sw = np.logical_or(jb, sw)
    
    jb_sw_df = df.loc[jb_sw]
    with open(outfp, 'a') as f:
        jb_sw_df.to_csv(f, index=False, header = f.tell() == 0)
os.remove(outfp)

In [90]:
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'jbswtest.tmp')
proj.get_sw_jb(infp, outfp)
df = pd.read_csv(outfp)
os.remove(outfp)
df.shape

(73, 31)

# Flight Delays to/from San Diego

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). 

The zip file at the [URL](https://s3-us-west-1.amazonaws.com/dsc80-sp19-data/project02/flight-delays.zip) contains a file `to_from_san.csv` that consists of all flights either to or from SAN (San Diego) in 2015 -- i.e. the output of Question 1. This dataset should match the dataset that your code returned in question 1.

Read in `to_from_san.csv` using `read_csv` and inspect the dataframe for an initial assessment about the data quality.

In [91]:
to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
flight_assess = pd.read_csv(to_from_san_filepath)

In [92]:
flight_assess.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Understanding the data types of the columns

**Question 2**:

* First, classify the *kind* of data each column contains. Create a function `data_kinds` of zero variables which outputs a (hard-coded) dictionary of data kinds, keyed by column name, with values `Q`, `O`, `N` (for 'Quantitative', 'Ordinal', or 'Nominal').

* Second, decide the best data *type* for each column. Create a function `data_types` of zero variables which outputs a (hard-coded) dictionary of data types, keyed by column name, with values `str`, `int`, `float`, `bool`. 

*Remark 1*: A column which *should* be `int`s, but contain `NaN`, *must* be a float column. See Lecture 2 notes an explanation of `NaN` and data-types.


In [94]:
kind_dict = {'YEAR': 'O', 
             'MONTH': 'O', 
             'DAY': 'O', 
             'DAY_OF_WEEK': 'O', 
             'AIRLINE': 'O', 
             'FLIGHT_NUMBER': 'N',
             'TAIL_NUMBER': 'N', 
             'ORIGIN_AIRPORT': 'N', 
             'DESTINATION_AIRPORT': 'N',
             'SCHEDULED_DEPARTURE': 'O',
             'DEPARTURE_TIME': 'Q', 
             'DEPARTURE_DELAY': 'Q', 
             'TAXI_OUT': 'O',
             'WHEELS_OFF': 'O',
             'SCHEDULED_TIME': 'O',
             'ELAPSED_TIME': 'Q', 
             'AIR_TIME': 'Q',
             'DISTANCE': 'Q', 
             'WHEELS_ON': 'O',
             'TAXI_IN': 'O',
             'SCHEDULED_ARRIVAL': 'O',
             'ARRIVAL_TIME': 'Q',
             'ARRIVAL_DELAY': 'Q', 
             'DIVERTED': 'N', 
             'CANCELLED': 'N', 
             'CANCELLATION_REASON': 'N',
             'AIR_SYSTEM_DELAY': 'Q', 
             'SECURITY_DELAY': 'Q', 
             'AIRLINE_DELAY': 'Q',
             'LATE_AIRCRAFT_DELAY': 'Q',
             'WEATHER_DELAY': 'Q'
            }

In [250]:
type_dict = {'YEAR': 'int', 
             'MONTH': 'int', 
             'DAY': 'int', 
             'DAY_OF_WEEK': 'int', 
             'AIRLINE': 'str', 
             'FLIGHT_NUMBER': 'int',
             'TAIL_NUMBER': 'str', 
             'ORIGIN_AIRPORT': 'str', 
             'DESTINATION_AIRPORT': 'str',
             'SCHEDULED_DEPARTURE': 'int', 
             'DEPARTURE_TIME': 'float', 
             'DEPARTURE_DELAY': 'float', 
             'TAXI_OUT': 'float',
             'WHEELS_OFF': 'float', 
             'SCHEDULED_TIME': 'int',
             'ELAPSED_TIME': 'float', 
             'AIR_TIME': 'float',
             'DISTANCE': 'int', 
             'WHEELS_ON': 'float', 
             'TAXI_IN': 'float',
             'SCHEDULED_ARRIVAL': 'int', 
             'ARRIVAL_TIME': 'float',
             'ARRIVAL_DELAY': 'float', 
             'DIVERTED': 'bool', 
             'CANCELLED': 'bool', 
             'CANCELLATION_REASON': 'str',
             'AIR_SYSTEM_DELAY': 'float', 
             'SECURITY_DELAY': 'float', 
             'AIRLINE_DELAY': 'float',
             'LATE_AIRCRAFT_DELAY': 'float',
             'WEATHER_DELAY': 'float'
            }

### Read in the typed flights data

Read in the flights data using your dictionary of data-types in `read_csv`. This both speeds up parsing, as well as gives you the correct data-types upon reading (which columns would pandas *parse incorrectly* if you didn't use a `dtype` dictionary?)

*Remark 2*: Use the dictionary of types from question 2 as a keyword argument in `read_csv`.


In [96]:
dtypes = proj.data_types()

flights = pd.read_csv(to_from_san_filepath, dtype=dtypes)

**Question 3 (Basic Stats):**

Define a function `basic_stats` that takes `flights` and outputs a dataframe that contains statistics for flights arriving/departing for SAN. That is, the output should have two rows, indexed by `ARRIVING` and `DEPARTING`, and have the following columns:

1. number of arriving/departing flights to/from SAN (`count`).
2. average flight (arrival) delay of arriving/departing flights to/from SAN (`mean`).
3. median flight (arrival) delay of arriving/departing flights to/from SAN (`median`).
4. the airline code of the airline that most often arrives/departs to/from SAN (`code`).
5. the proportion of arriving/departing flights to/from SAN that are canceled (`canceled`).
6. the airline code of the airline with the longest flight (arrival) delay among all flights arriving/departing to/from SAN (`max`).

*Remark:* Null values should not be considered when computing statistics; however, think about whether e.g. the average flight delay is likely higher or lower that the "true mean" by making this choice.

*Hint*: Explore the different types of dictionaries you can pass to `aggregate`.

In [284]:
arriving_df = flights.loc[flights['DESTINATION_AIRPORT'] == 'SAN']
departing_df = flights.loc[flights['ORIGIN_AIRPORT'] == 'SAN']

count = [len(arriving_df), len(departing_df)]
mean = [arriving_df['ARRIVAL_DELAY'].mean(), departing_df['ARRIVAL_DELAY'].mean()]
median = [arriving_df['ARRIVAL_DELAY'].median(), departing_df['ARRIVAL_DELAY'].median()]
code = [arriving_df['AIRLINE'].value_counts().keys()[0], departing_df['AIRLINE'].value_counts().keys()[0]]
canceled = [arriving_df['CANCELLED'].sum() / len(arriving_df), departing_df['CANCELLED'].sum() / len(departing_df)]
mx = [arriving_df.loc[arriving_df['ARRIVAL_DELAY'] == arriving_df['ARRIVAL_DELAY'].max(), 'AIRLINE'].iloc[0],
       departing_df.loc[departing_df['ARRIVAL_DELAY'] == departing_df['ARRIVAL_DELAY'].max(), 'AIRLINE'].iloc[0]]

data = {'count': count,
        'mean': mean,
        'median': median,
        'code': code,
        'canceled': canceled,
        'max': mx}

df = pd.DataFrame(data=data, index = ['ARRIVING', 'DEPARTING'])
df

,count,mean,median,code,canceled,max
ARRIVING,70207,3.676826,-4.0,WN,0.009671,AA
DEPARTING,70207,3.328988,-5.0,WN,0.011181,AA


### Understanding flight delays: Departures, Arrivals, and everything in-between

**Question 4**

Often `DEPARTURE_DELAY` is thought to be the main cause of a flight delay -- i.e., when the flight is late pushing off from the gate. 

However, there are other ways that flights can be late: waiting on the tarmac, headwinds, turbulence, circling a busy airport, and waiting for a gate after landing. First, we will analyze all the ways in which a flight can be delayed.

* First, create a function `depart_arrive_stats` that takes in a dataframe like `flights` and calculates the following quantities in a series, indexed by `late1`, `late2`, `late3`:
    - The proportion of flights from/to SAN that leave late, but arrive early or on-time (`late1`).
    - The proportion of flights from/to SAN that leaves early, or on-time, but arrives late (`late2`).
    - The proportion of flights from/to SAN that both left late and arrived late (`late3`).
    
* Second, create a function `depart_arrive_stats_by_airline` that takes in a dataframe like `flights` and calculates the quantities above broken down by *airline*. That is, the output is a dataframe, indexed by `AIRLINE`, with columns given by `late1`, `late2`, `late3`.

In [98]:
late1 = np.logical_and(flights['DEPARTURE_DELAY'] > 0, flights['ARRIVAL_DELAY'] <= 0).sum() / len(flights)
late2 = np.logical_and(flights['DEPARTURE_DELAY'] <= 0, flights['ARRIVAL_DELAY'] > 0).sum() / len(flights)
late3 = np.logical_and(flights['DEPARTURE_DELAY'] > 0, flights['ARRIVAL_DELAY'] > 0).sum() / len(flights)
pd.Series([late1, late2, late3], index=['late1', 'late2', 'late3'])

late1    0.119853
late2    0.089329
late3    0.278804
dtype: float64

In [99]:
airline_group = flights.groupby('AIRLINE')
airline_group.apply(proj.depart_arrive_stats)

,late1,late2,late3
AIRLINE,,,
AA,0.107658,0.104986,0.219363
AS,0.060712,0.159467,0.145359
B6,0.146720,0.128827,0.248111
DL,0.128545,0.108387,0.153656
F9,0.067024,0.098525,0.270107
HA,0.118263,0.288922,0.264970
NK,0.045812,0.149092,0.270805
OO,0.049317,0.119447,0.269960
UA,0.213901,0.045404,0.286262


### Flight delays and day of the week

**Question 5**

Next, we'd like to understand the flight traffic to/from SAN by day of the week. Day of the week is specified by integers 1 through 7; verify for yourself which integer corresponds to which day (hint: you have the *date* for each flight as well!).

Next create two functions to understand both the amount of traffic and the average flight delay of flights for each airline by day-of-the week. We both want to understand *presence* each airline has as well as their performance.

1. Create a function `cnts_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe with 
    - a column for each distinct value of `AIRLINE`,
    - a row for each day of the week, and
    - entries that give the total number of flights that airline has on that day of the week over 2015.
2. Create a function `mean_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe with 
    - a column for each distinct value of `AIRLINE`,
    - a row for each day of the week, and
    - entries that give the average `ARRIVAL_DELAY` for the flights of each airline on that day of the week.
    
*Hint:* Both `groupby` and `pivot` should be useful here!

Your output should have the *form* of the table below (not the entries!)

<img src="pivot.png" />

In [100]:
pd.pivot_table(flights, values = 'YEAR', columns='AIRLINE', index='DAY_OF_WEEK', aggfunc= 'count')

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,1915,1564,384,1753,218,96,531,1859,2369,495,536,9238
2,1872,1345,359,1647,210,96,529,1829,2211,463,544,9300
3,1922,1428,359,1719,227,96,530,1814,2293,472,542,9328
4,1888,1551,380,1729,220,96,531,1835,2328,510,531,9108
5,1859,1449,376,1701,208,94,518,1813,2279,505,523,9006
6,1755,1476,270,1433,195,94,518,1837,1743,496,365,7201
7,1886,1465,387,1726,214,96,532,1889,2172,503,443,8587


In [101]:
pd.pivot_table(flights, values = 'ARRIVAL_DELAY', columns='AIRLINE', index='DAY_OF_WEEK', aggfunc= np.mean)

AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,2.245981,-2.652286,0.928571,-3.096904,3.233945,5.842105,9.222656,7.405257,3.081280,1.764344,12.804878,6.355997
2,0.791037,-4.535474,4.507123,-2.988408,4.975728,7.385417,10.669903,5.109574,5.617431,1.354626,4.645522,5.394194
3,1.811808,-4.030899,2.735043,-2.437098,5.634361,16.833333,6.304264,5.566741,3.686344,-2.145299,0.631970,3.556762
4,4.076552,-2.180995,3.152406,-2.339721,4.531818,7.552083,7.931298,7.971901,4.957124,1.198397,8.277040,6.582300
5,1.235679,-1.040887,3.329759,-5.699528,4.830918,8.021505,7.242718,5.920156,1.756997,3.017928,9.325536,7.961504
6,-0.427912,-3.283673,-1.854478,-3.608726,0.119171,7.159574,6.943026,3.746961,-1.338924,-2.227926,0.101370,1.684625
7,1.611924,-2.318213,2.422043,-4.857392,6.112150,5.229167,8.754753,8.570806,-0.190121,2.257545,12.594533,6.867543


### Understanding null values in the flights data

**Question 6 (Missing by Design)**

Now we would like to understand how data is missing in the flights data. First, compute the proportion of each column of `flights` which are non-null. Do not turn this in, but it will be useful information in doing the next few problems.

Recall that a column is *missing by design* if you can determine when the entry of a column is missing based solely on other data in the same row. That is, there is *no randomness* in determining when an entry is missing.

if you can describe when the column is missing a value with a logical (not random) condition. That is, you can express which rows will have missing values in terms of logical statements about the *other* columns in the same row.

* The column `ARRIVAL_DELAY` is *missing by design*. Create a function `predict_null_arrival_delay` that doesn't depend on the values of `ARRIVAL_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `ARRIVAL_DELAY` is null; otherwise it returns `False`.
    - Since the function doesn't depend on `ARRIVAL_DELAY`, it should work on a row even if the `ARRIVAL_DELAY` index is dropped.
    - You can check your function by using `flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null, axis=1)` and compare it to the `ARRIVAL_DELAY` column!


* The column `AIRLINE_DELAY` is *missing by design*. As above, create a function `predict_null_airline_delay` that doesn't depend on the values of `AIRLINE_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `AIRLINE_DELAY` is null; otherwise it returns `False`.


In [102]:
def predict_null_arrival_delay(row):
    if row['DIVERTED'] == True or row['CANCELLED'] == True:
        return True
    return False

In [103]:
def predict_null_airline_delay(row):
    if predict_null_arrival_delay(row) or row['ARRIVAL_DELAY'] < 15:
        return True
    return False

In [277]:
flights['TAIL_NUMBER'].isnull().sum()

209

**Question 7 (Missingness Types)**

Now we'd like to determine missingness of the column `DEPARTURE_DELAY`. In particular, we'd like to perform a permutation test to determine the missingness of `DEPARTURE_DELAY` dependent on the column `AIRLINE`.

* Create a function `perm4missing`:
    - that takes in `flights`, a column `col`, and a number `N` and 
    - returns the p-value of the test (using `N` simulations) that determines if `DEPARTURE_DELAY` is MAR dependent on `col`. That is `perm4missing(flights, 'AIRLINE', N)` should return the p-value for the test above.
    - *Remark*: to help your work, create helper functions whose output you can plot, to assess the correctness of your p-value!
    
* Use your function above to determine the columns `col` for which "`DEPARTURE_DELAY` is MAR dependent on `col`" using a significance level of 0.01. Only consider the categorical columns `YEAR`,`DAY_OF_WEEK`, `AIRLINE`,`DIVERTED`, `CANCELLATION_REASON`. Return your answer in a (hard-coded) list returned by a function called `dependent_cols`.

* Create a function `missing_types` of zero variables, which:
    - Returns a Series, indexed by the following columns of `flights`: `CANCELLED`, `CANCELLATION_REASON`, `TAIL_NUMBER`, `ARRIVAL_TIME`.
    - The values should contain the most-likely missingness type of each column. 
    - The values of this Series should be `MD, MCAR, MAR, MNAR, NaN` (use `NaN` if there are no missing values). 


In [104]:
def get_tvd(df, col):
    pivot = (
        df
        .pivot_table(index='is_null', columns=col, aggfunc='size')
        .apply(lambda x:x / x.sum(), axis=1)
    )
    return pivot.diff().iloc[-1].abs().sum() / 2

def sim_null(flights, col):
    shuffled_col = (
        flights[col]
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    shuffled = (
        flights
        .assign(**{
            col: shuffled_col, 
            'is_null': flights['DEPARTURE_DELAY'].isnull()
        })
    )
    
    return get_tvd(shuffled, col)

def perm4missing(flights, col, N):
    obs_flights = flights.assign(is_null = flights['DEPARTURE_DELAY'].isnull())
    observed_tvd = get_tvd(obs_flights, col)
    
    tvds = []
    for _ in range(N):
        tvd = sim_null(flights, col)
        tvds.append(tvd)
    
    pval = np.mean(tvds > observed_tvd)
    
    return (tvds, pval)

proj.perm4missing(flights, 'AIRLINE', 100)

0.0

In [105]:
print(
    proj.perm4missing(flights, 'YEAR', 100),
    proj.perm4missing(flights, 'DAY_OF_WEEK', 100),
    proj.perm4missing(flights, 'AIRLINE', 100),
    proj.perm4missing(flights, 'DIVERTED', 100),
    proj.perm4missing(flights, 'CANCELLATION_REASON', 100)
)

0.0 0.0 0.0 0.34 0.92


In [276]:
pd.Series(
    {'CANCELLED': np.NaN,
     'CANCELLATION_REASON': 'MD',
     'TAIL_NUMBER': 'MCAR',
     'ARRIVAL_TIME': 'MAR'
    })

CANCELLED              NaN
CANCELLATION_REASON     MD
TAIL_NUMBER            NaN
ARRIVAL_TIME           MAR
dtype: object

# Simpson's Paradox: JetBlue vs Southwest

The remainder of the questions investigates the presence of Simpson's paradox in the flights dataset. Read through Lecture 05 for a summary of Simpson's Paradox and related links.

The csv file `southwest_vs_jetblue.csv` contains all Southwest and JetBlue flights in 2015.

In this dataset, we are going to verify the following occurrences of Simpson's Paradox: For a given set of airports,
* The average departure delay of Southwest is greater than (or less than) the average departure delay of JetBlue.
* Airport by airport, the average departure delay of Southwest is *less* than (or greater than) the average departure delay of JetBlue.

That is, the inequalities of the average flight delays between the two airlines are reversed when viewed at the level of each airport. In fact this reversal holds for *every* airport being considered.

In [106]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

# The `usecols` keyword:
# choose *only* the columns you need to reduce your memory footprint!
usecols = ['AIRLINE', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY']

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype, usecols=usecols)

**Question 8**

Filter the dataset `jb_sw` to flights *originating* from the following 10 airports: ABQ, BDL, BUR, DCA, MSY, PBI, PHX, RNO, SJC, SLC.

Illustrate Simpson's paradox with this table:
* Calculate the proportion of each airline's flights that are delayed:
    - Create a function `prop_delayed_by_airline` that takes in a dataframe like `jb_sw` and returns a DataFrame indexed by airline that contains the proportion of each airline's flights that are delayed.
* Calculate these proportions across all airports in the dataset:
    - Create a function `prop_delayed_by_airline_airport` that takes in a dataframe like `jb_sw` and returns a DataFrame, with columns given by airports, indexed by airline, that contains the proportion of each airline's flights that are delayed at each airport.

*Remark:* For the purpose of this question, a canceled flight is **not** delayed.

Verify that Simpson's paradox is present in this output! 

Can you explain *why* Simpson's paradox is occurring? (Hint: where are these airports located? Which have the most flights?)

In [263]:
jb_sw = jb_sw.fillna(0)
jb_sw.groupby('AIRLINE').agg({'DEPARTURE_DELAY': lambda x: (x > 0).sum() / len(x)})

,DEPARTURE_DELAY
AIRLINE,
B6,0.382182
WN,0.449186


In [264]:
def origin_airport(row):
    airports = ['ABQ', 'BDL', 'BUR', 'DCA', 'MSY', 'PBI', 'PHX', 'RNO', 'SJC', 'SLC']
    if row['ORIGIN_AIRPORT'] in airports:
        return True
    return False
airport_df = jb_sw.loc[jb_sw.apply(origin_airport, axis = 1)]

In [265]:
d = (airport_df.groupby(['AIRLINE', 'ORIGIN_AIRPORT'])
      .agg({'DEPARTURE_DELAY': lambda x: (x > 0).sum() / len(x)})
      .pivot_table(index='AIRLINE', columns='ORIGIN_AIRPORT', values='DEPARTURE_DELAY')
     )
d

ORIGIN_AIRPORT,ABQ,BDL,BUR,DCA,MSY,PBI,PHX,RNO,SJC,SLC
AIRLINE,,,,,,,,,,
B6,0.489051,0.371207,0.449102,0.285039,0.391111,0.476827,0.576169,0.544379,0.518887,0.485859
WN,0.457300,0.367560,0.342187,0.284563,0.377690,0.428684,0.549186,0.377978,0.464572,0.441616


**Question 9**

Your work above illustrates Simpson's paradox on the specific dataset of flights originating from 10 specific airports. However, this still requires you to look at two dataframe to see if the paradox is present. Now, you will create a function that verifies Simpson's paradox in general. You will do this by writing code to compare the two dataframes, instead of inspecting them manually.

Create a function `verify_simpson` that returns a boolean output regarding if the paradox is present.
```
verify_simpson(df, group1, group2, occur)
```
- df is a dataframe (e.g. jb_sw),
- group1 is the first group being aggregated against (e.g. `AIRLINE`),
- group2 is the second group being aggregated against (e.g. `ORIGIN_AIRPORT`),
- occur is a column with values {0, 1}, denoting if an event occurred for that individual.
  (e.g. "1 if flight was delayed" and "0 if flight was not delayed")

`verify_simpson` should return `True` only if there is a reversal for *every* value of `group2` (e.g. for every airport).

Verify that you function works on the previous question!

In [272]:
def verify_simpson(df, group1, group2, occur):
    first = df.groupby(group1).agg({occur: lambda x: (x > 0).sum() / len(x)})
    lower_index = first[occur].idxmin()
    higher_index = first[occur].idxmax()
    
    second = (df.groupby([group1, group2])
      .agg({occur: lambda x: (x > 0).sum() / len(x)})
      .pivot_table(index=group1, columns=group2, values=occur)
     )
    
    return (second.loc[higher_index] < second.loc[lower_index]).all()

In [286]:
verify_simpson(airport_df, 'AIRLINE', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY')

True

In [271]:
df = pd.DataFrame([[4,2,1], [1,2,0], [1,4,0], [4,4,1]], columns=[0,1,2])
verify_simpson(df, 0, 1, 2)

False

### Extra Credit

The extra credit question(s) will be **due at the end of the quarter**, however it is often easier to do these questions while working on the assignment itself -- so do give it a try now!

**Question 10 (Searching for Simpson's Paradox):**

As you observed from the reading in the lecture notes, Simpson's Paradox often occurs due to some confounding factor among the columns of a dataset. In the case of gender balance in student admissions to academic departments at UC Berkeley, the confounding factor was the admission rate (i.e. how hard it is to gain admission to a department).

What might be a confounding factor be for flight delays among airports in question 8? Now you are going to write code to discover instances of Simpson's Paradox; that is, you will programmatically find interesting relationships present in the data.

Given the dataset `jb_sw`, we'd like to find new groups of airports, as in question 8, for which the statistics of flight delays between JetBlue and Southwest satisfy Simpson's Paradox.

Create a function `search_simpsons` that takes in the `jb_sw` dataset and a number `N`, and returns a list of `N` airports for which the proportion of flight delays between JetBlue and Southwest satisfies Simpson's Paradox.
- Only consider airports that have '3 letter codes',
- Only consider airports that have at least one JetBlue *and* Southwest flight.

*Remark 1:* Iterate through groups of airports of size `N` using `itertools.combinations` until you find a group that works. Make sure your function finishes, even if it doesn't find something.

*Remark 2:* You should be using your work from Question 9!

In [290]:
jb_sw_filepath = os.path.join('data', 'jetblue_or_sw.csv')
dtype = proj.data_types()

jb_sw = pd.read_csv(jb_sw_filepath, dtype=dtype)

In [291]:
jb_sw

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,B6,304,N607JB,SJU,JFK,155,...,501.0,11.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,B6,1990,N597JB,SJU,EWR,206,...,516.0,4.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,B6,668,N653JB,PSE,MCO,255,...,451.0,-9.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,520.0,20.0,False,False,NaN,20.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,B6,262,N627JB,SJU,BOS,330,...,619.0,-16.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,B6,2134,N307JB,SJU,MCO,400,...,730.0,85.0,False,False,NaN,0.0,0.0,85.0,0.0,0.0
6,2015,1,1,4,B6,730,N621JB,BQN,MCO,419,...,606.0,-7.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,B6,768,N317JB,PSE,MCO,424,...,629.0,-1.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,1,1,4,B6,2276,N646JB,SJU,BDL,438,...,908.0,89.0,False,False,NaN,17.0,0.0,72.0,0.0,0.0
9,2015,1,1,4,B6,2001,N358JB,BUF,JFK,535,...,648.0,-15.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project01.py
* Be sure your free repsonse questions are all answered, readable, and that you haven't changed the cells outside the horizontal lines!

### To submit:
* **Convert the notebook to PDF and upload to gradescope for grading the free response.**
* **Upload the .py file to gradescope**